# Import libraries

In [52]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Set global data path

In [28]:
# path where the csv files are saved
data_folder = '/Users/luisalcala/Desktop/redbull/data'

# Load raw data

In [29]:
# dictionaty to store the standings
standings_files = {}

# for each file in the path (we only have csv files)
for file_name in os.listdir(data_folder):
    print(file_name)
    if file_name[0] == 's':
        # read csv with particular header
        df = pd.read_csv(os.path.join(data_folder, file_name), encoding='ISO-8859-1')
        # Add the DataFrame to the dictionary with key = file name
        standings_files[os.path.splitext(file_name)[0]] = df
    else:
        # read transfer csv
        transfers_df = pd.read_csv(os.path.join(data_folder, file_name), index_col=[0])

standings-bundesliga-19-20.csv
clean_full_standings.csv
standings-bundesliga-21-22.csv
transfers-bundesliga.csv
standings-bundesliga-23-24.csv
standings-bundesliga-22-23.csv
standings-bundesliga-20-21.csv


In [30]:
len(standings_files)

5

In [31]:
transfers_df.head()

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_euro,transfer_fee_million,...,to.club_name,to.competition_id,to.competition_name,date_of_birth_datetime,citizenship,position.main,position.other,height_cm,foot,updated_at
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,NaN,...,Holstein Kiel,L2,2. Bundesliga,1997-10-11,['Germany' 'Nigeria'],Left-Back,['Right-Back' 'Left Winger'],191.0,right,2024-02-25T02:16:17.122733
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.0,...,SC Paderborn,L1,Bundesliga,1995-01-30,['Germany'],Attacking Midfield,['Right Winger' 'Left Winger'],183.0,left,2024-02-24T10:13:57.036699
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,NaN,...,Carl Zeiss Jena,L3,3. Liga,1997-01-09,['Germany'],Defensive Midfield,['Central Midfield'],187.0,left,2024-02-24T10:13:48.186890
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,NaN,...,Magdeburg,L3,3. Liga,1997-04-19,['Germany'],Defensive Midfield,['Central Midfield' 'Left-Back'],182.0,left,2024-02-24T10:13:43.016587
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,NaN,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466


In [32]:
standings_files['standings-bundesliga-19-20'].head()

,Unnamed: 0,Unnamed: 1,Home,Unnamed: 3,Unnamed: 4,Unnamed: 5,Away,Unnamed: 7,Unnamed: 8,Unnamed: 9,Total,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,P,W,D,L,P,W,D,L,P,W,D,L,F,A,+/-,Pts
1,C,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
2,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
3,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
4,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65


# Standings Preprocessing

## Process column names

In [33]:
# as all stadings files have the same structure, we can set the column names we'd like
column_names = [
    'final_standing', 'team_name', 
    'home_points', 'home_wins', 'home_draws', 'home_losses', 
    'away_points', 'away_wins', 'away_draws', 'away_losses', 
    'total_points', 'total_wins', 'total_draws', 'total_losses',
    'goals_in_favor', 'goals_against', 'goals_balance', 'points'
    ]

# change column names for the standings files
for file_name, df in standings_files.items():
    df.columns = column_names
    df.drop(index=df.index[0], inplace=True)
    df.reset_index(drop=True, inplace=True)

In [34]:
standings_files['standings-bundesliga-19-20'].head(18)

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,C,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63
5,6,1899 Hoffenheim,17,7,1,9,17,8,6,3,34,15,7,12,53,53,0,52
6,7,VfL Wolfsburg,17,4,7,6,17,9,3,5,34,13,10,11,48,46,+2,49
7,8,SC Freiburg,17,9,2,6,17,4,7,6,34,13,9,12,48,47,+1,48
8,9,Eintracht Frankfurt,17,8,4,5,17,5,2,10,34,13,6,15,59,60,-1,45
9,10,Hertha Berlin,17,6,3,8,17,5,5,7,34,11,8,15,48,59,-11,41


In [35]:
standings_files['standings-bundesliga-20-21'].head(18)

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,C,Bayern Munich,17,13,4,0,17,11,2,4,34,24,6,4,99,44,+55,78
1,2,RB Leipzig,17,11,4,2,17,8,4,5,34,19,8,7,60,32,+28,65
2,3,Borussia Dortmund,17,11,2,4,17,9,2,6,34,20,4,10,75,46,+29,64
3,4,VfL Wolfsburg,17,10,4,3,17,7,6,4,34,17,10,7,61,37,+24,61
4,5,Eintracht Frankfurt,17,10,7,0,17,6,5,6,34,16,12,6,69,53,+16,60
5,6,Bayer 04 Leverkusen,17,8,5,4,17,6,5,6,34,14,10,10,53,39,+14,52
6,7,1. FC Union Berlin,17,8,8,1,17,4,6,7,34,12,14,8,50,43,+7,50
7,8,Borussia Mönchengladbach,17,8,4,5,17,5,6,6,34,13,10,11,64,56,+8,49
8,9,VfB Stuttgart,17,5,6,6,17,7,3,7,34,12,9,13,56,55,+1,45
9,10,SC Freiburg,17,7,6,4,17,5,3,9,34,12,9,13,52,52,0,45


## Define constant vars

In [36]:
N_TEAMS_SEASON = 18

## Change C and R values per corresponding numerical standing

In some files, the Champions and the Relegation teams are marked as C and R respectively, let's change that to 1 and 17/18 respectively. To do that, we just set the final standing as the index value +1 as we know the first is going to be the champion and the last two relegated (does not change over the seasons for now). 

In [37]:
for file_name, df in standings_files.items():
    df['final_standing'] = df.index + 1

In [38]:
standings_files['standings-bundesliga-19-20'].head(18)

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63
5,6,1899 Hoffenheim,17,7,1,9,17,8,6,3,34,15,7,12,53,53,0,52
6,7,VfL Wolfsburg,17,4,7,6,17,9,3,5,34,13,10,11,48,46,+2,49
7,8,SC Freiburg,17,9,2,6,17,4,7,6,34,13,9,12,48,47,+1,48
8,9,Eintracht Frankfurt,17,8,4,5,17,5,2,10,34,13,6,15,59,60,-1,45
9,10,Hertha Berlin,17,6,3,8,17,5,5,7,34,11,8,15,48,59,-11,41


In [39]:
standings_files['standings-bundesliga-20-21'].head(18)

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,1,Bayern Munich,17,13,4,0,17,11,2,4,34,24,6,4,99,44,+55,78
1,2,RB Leipzig,17,11,4,2,17,8,4,5,34,19,8,7,60,32,+28,65
2,3,Borussia Dortmund,17,11,2,4,17,9,2,6,34,20,4,10,75,46,+29,64
3,4,VfL Wolfsburg,17,10,4,3,17,7,6,4,34,17,10,7,61,37,+24,61
4,5,Eintracht Frankfurt,17,10,7,0,17,6,5,6,34,16,12,6,69,53,+16,60
5,6,Bayer 04 Leverkusen,17,8,5,4,17,6,5,6,34,14,10,10,53,39,+14,52
6,7,1. FC Union Berlin,17,8,8,1,17,4,6,7,34,12,14,8,50,43,+7,50
7,8,Borussia Mönchengladbach,17,8,4,5,17,5,6,6,34,13,10,11,64,56,+8,49
8,9,VfB Stuttgart,17,5,6,6,17,7,3,7,34,12,9,13,56,55,+1,45
9,10,SC Freiburg,17,7,6,4,17,5,3,9,34,12,9,13,52,52,0,45


## Join in one dataframe by year

For each standing file, we set the season id based on the file name to match the season id of the transfers file in case we need to cross data later.

In [40]:
standings_list = []

for file_name, df in standings_files.items():
    # set season id with the file name
    df['season_id'] = '20'+file_name[len(file_name)-5:len(file_name)-3]
    standings_list.append(df)

# concat all standings in a single dataframe
standings_df = pd.concat(standings_list, ignore_index=True)
print(standings_df.shape)
standings_df.head()

(90, 19)


,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82,2019
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69,2019
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66,2019
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65,2019
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63,2019


In [41]:
standings_df.sort_values(by=['season_id', 'final_standing'], inplace=True)

## Take out + sign on goals

In [42]:
standings_df['goals_balance'] = standings_df['goals_balance'].str.replace('+', '', regex=False)
standings_df.head()

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,68,82,2019
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,43,69,2019
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,44,66,2019
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,26,65,2019
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,17,63,2019


## Set correct data types

In [43]:
standings_df.columns

Index(['final_standing', 'team_name', 'home_points', 'home_wins', 'home_draws',
       'home_losses', 'away_points', 'away_wins', 'away_draws', 'away_losses',
       'total_points', 'total_wins', 'total_draws', 'total_losses',
       'goals_in_favor', 'goals_against', 'goals_balance', 'points',
       'season_id'],
      dtype='object')

In [44]:
standings_df.dtypes

final_standing     int64
team_name         object
home_points       object
home_wins         object
home_draws        object
home_losses       object
away_points       object
away_wins         object
away_draws        object
away_losses       object
total_points      object
total_wins        object
total_draws       object
total_losses      object
goals_in_favor    object
goals_against     object
goals_balance     object
points            object
season_id         object
dtype: object

In [45]:
# set numerical columns
numerical_cols = ['final_standing', 'home_points', 'home_wins', 'home_draws',
       'home_losses', 'away_points', 'away_wins', 'away_draws', 'away_losses',
       'total_points', 'total_wins', 'total_draws', 'total_losses',
       'goals_in_favor', 'goals_against', 'goals_balance', 'points']

# set categorical columns
cat_cols = ['team_name', 'season_id']

In [46]:
standings_df[numerical_cols] = standings_df[numerical_cols].apply(pd.to_numeric, errors='coerce')

In [47]:
standings_df.dtypes

final_standing     int64
team_name         object
home_points        int64
home_wins          int64
home_draws         int64
home_losses        int64
away_points        int64
away_wins          int64
away_draws         int64
away_losses        int64
total_points       int64
total_wins         int64
total_draws        int64
total_losses       int64
goals_in_favor     int64
goals_against      int64
goals_balance      int64
points             int64
season_id         object
dtype: object

In [48]:
standings_df.tail()

,final_standing,team_name,home_points,home_wins,home_draws,home_losses,away_points,away_wins,away_draws,away_losses,total_points,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
49,14,Borussia Mönchengladbach,17,5,6,6,17,2,7,8,34,7,13,14,56,67,-11,34,2023
50,15,1 FC Union Berlin,17,7,2,8,17,2,4,11,34,9,6,19,33,58,-25,33,2023
51,16,VfL Bochum,17,5,8,4,17,2,4,11,34,7,12,15,42,74,-32,33,2023
52,17,1 FC Köln,17,4,4,9,17,1,8,8,34,5,12,17,28,60,-32,27,2023
53,18,Darmstadt 98,17,1,3,13,17,2,5,10,34,3,8,23,30,86,-56,17,2023


## Clean team names

In [49]:
d_team_name_correction = {
    '1. FC Union Berlin': 'FC Union Berlin',
    '1 FC Union Berlin': 'FC Union Berlin',
    '1. FC Köln': 'FC Köln',
    '1 FC Köln': 'FC Köln',
    '1 FC Heidenheim': 'FC Heidenheim'
}

standings_df['team_name'] = standings_df['team_name'].replace(d_team_name_correction)

## Save changes to a new csv

In [50]:
standings_df.to_csv(os.path.join(data_folder, 'clean_full_standings.csv'), index=False)

# Transfers Preprocessing

In [51]:
transfers_df.head()

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_euro,transfer_fee_million,...,to.club_name,to.competition_id,to.competition_name,date_of_birth_datetime,citizenship,position.main,position.other,height_cm,foot,updated_at
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,NaN,...,Holstein Kiel,L2,2. Bundesliga,1997-10-11,['Germany' 'Nigeria'],Left-Back,['Right-Back' 'Left Winger'],191.0,right,2024-02-25T02:16:17.122733
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.0,...,SC Paderborn,L1,Bundesliga,1995-01-30,['Germany'],Attacking Midfield,['Right Winger' 'Left Winger'],183.0,left,2024-02-24T10:13:57.036699
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,NaN,...,Carl Zeiss Jena,L3,3. Liga,1997-01-09,['Germany'],Defensive Midfield,['Central Midfield'],187.0,left,2024-02-24T10:13:48.186890
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,NaN,...,Magdeburg,L3,3. Liga,1997-04-19,['Germany'],Defensive Midfield,['Central Midfield' 'Left-Back'],182.0,left,2024-02-24T10:13:43.016587
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,NaN,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466
